<a href="https://colab.research.google.com/github/andreplacido/Beam_colab_demo/blob/master/05_composite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!{'pip install --quiet apache_beam'}

     |████████████████████████████████| 8.6MB 3.4MB/s 
     |████████████████████████████████| 153kB 47.9MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 1.4MB 40.6MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 71kB 6.0MB/s 
     |████████████████████████████████| 63.8MB 110kB/s 
     |████████████████████████████████| 829kB 48.0MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 112kB 36.8MB/s 
ERROR: pydrive 1.3.1 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
ERROR: multiprocess 0.70.10 has requirement dill>=0.3.2, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [5]:
!{'mkdir -p data'}

In [3]:
from google.colab import files
uploaded = files.upload()

Saving dept_data.txt to dept_data.txt


In [6]:
!ls

data  dept_data.txt  sample_data


In [16]:
# otimização do código criando uma class que concentrar a(s) funções comuns que serão aplicadas em mais de um ptransform
# ref 02_Filter repare que para Account e HR se repetem passos de transformação no código
import apache_beam as beam

class MyTransform(beam.PTransform):
  
  def expand(self, input_coll):
    
    a = ( 
        input_coll
                       | 'Group and sum1' >> beam.CombinePerKey(sum)
                       | 'count filter accounts' >> beam.Filter(filter_on_count)
                       | 'Regular accounts employee' >> beam.Map(format_output)
              
    )
    return a

def SplitRow(element):
    return element.split(',')
  
  
def filter_on_count(element):
  name, count = element
  if count > 30:
    return element
  
def format_output(element):
  name, count = element
  return ', '.join((name.encode('ascii'),str(count),'Regular employee'))

p = beam.Pipeline()

input_collection = ( 
                      p 
                      | "Read from text file" >> beam.io.ReadFromText('dept_data.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )

accounts_count = (
                      input_collection
                      | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
                      | 'Pair each accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " +record[1], 1))
                      | 'composite accounts' >> MyTransform()
                      | 'Write results for account' >> beam.io.WriteToText('data/Account')
                 )

hr_count = (
                input_collection
                | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
                | 'Pair each hr employee with 1' >> beam.Map(lambda record: ("HR, " +record[1], 1))
                | 'composite HR' >> MyTransform()
                | 'Write results for hr' >> beam.io.WriteToText('data/HR')
           ) 
p.run()
  
# Sample the first 20 results, remember there are no ordering guarantees.
!{('head -n 20 data/Account-00000-of-00001')}
!{('head -n 20 data/HR-00000-of-00001')}

(b'Accounts, Marco', '31', 'Regular employee')
(b'Accounts, Rebekah', '31', 'Regular employee')
(b'Accounts, Itoe', '31', 'Regular employee')
(b'Accounts, Edouard', '31', 'Regular employee')
(b'Accounts, Kyle', '62', 'Regular employee')
(b'Accounts, Kumiko', '31', 'Regular employee')
(b'Accounts, Gaston', '31', 'Regular employee')
(b'HR, Beryl', '62', 'Regular employee')
(b'HR, Olga', '31', 'Regular employee')
(b'HR, Leslie', '31', 'Regular employee')
(b'HR, Mindy', '31', 'Regular employee')
(b'HR, Vicky', '31', 'Regular employee')
(b'HR, Richard', '31', 'Regular employee')
(b'HR, Kirk', '31', 'Regular employee')
(b'HR, Kaori', '31', 'Regular employee')
(b'HR, Oscar', '31', 'Regular employee')
